In [ ]:
import sys
from pathlib import Path
# Caminho raiz do projeto
projeto_raiz = Path("D:/Baixados/github/imaginai-TCC/ImaginAI")
# Adiciona o caminho da pasta 'codigo' ao sys.path
sys.path.append(str(projeto_raiz / "codigo"))
import carregadores.model_loader as model_loader
import fluxo.pipeline as pipeline
from PIL import Image
from transformers import CLIPTokenizer
import torch



# Configuração do dispositivo
dispositivo = "cpu"
PERMITIR_CUDA = False
PERMITIR_MPS = False

if torch.cuda.is_available() and PERMITIR_CUDA:
    dispositivo = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and PERMITIR_MPS:
    dispositivo = "mps"

print(f"Usando o dispositivo: {dispositivo}")

# Verificação do dispositivo para garantir que seja compatível com o Torch Generator
if dispositivo not in ["cpu", "cuda", "mps"]:
    raise ValueError(f"Dispositivo '{dispositivo}' não é compatível com o PyTorch.")

# Ajusta os caminhos relativos para arquivos necessários
base_path = Path(r"D:\Baixados\github\imaginai-TCC\ImaginAI\banco")
model_path = Path(r"D:\Baixados\github\imaginai-TCC\ImaginAI\modelo_treinado")
vocab_path = base_path / "vocab.json"
merges_path = base_path / "merges.txt"
arquivo_modelo_treinado = model_path / "v1-5-pruned-emaonly.ckpt"

# Carregando o tokenizer e os modelos
gerar_token = CLIPTokenizer(str(vocab_path), merges_file=str(merges_path))
modelos = model_loader.carregar_modelos_pesos_padrao(str(arquivo_modelo_treinado), dispositivo)

# Configurando o gerador de semente
gerador = torch.Generator(device=dispositivo)

# Parâmetros de texto para imagem
prompt = "apple"
uncond_prompt = ""  # Também conhecido como prompt negativo
fazer_cfg = True
escala_cfg = 8  # mínimo: 1, máximo: 14

# Parâmetros de imagem para imagem
imagem_entrada = None
forca = 0.9

# Configuração do amostrador
nome_denoiser = "ddpm"
num_inferencia_passos = 10
semente = 42

# Geração da imagem
imagem_saida = pipeline.gerar(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    imagem_entrada=imagem_entrada,
    forca=forca,
    fazer_cfg=fazer_cfg,
    escala_cfg=escala_cfg,
    nome_denoiser=nome_denoiser,
    n_inferencia_passos=num_inferencia_passos,
    semente=semente,
    modelos=modelos,
    dispositivo=dispositivo,
    dispositivo_ocioso="cpu",
    gerar_token=gerar_token,
)

# Combine a imagem de entrada e a imagem de saída em uma única imagem.
output_image = Image.fromarray(imagem_saida)
output_image.show()  # Abre a imagem gerada
